In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#clear output
from IPython.display import clear_output
import threading
import time
from collections import deque

In [2]:
data_dir="C:\\Users\\noush\\Downloads\\Eating\\"

fn="Dain Eating 1.mp4"
file_path=data_dir+fn

In [3]:
class VideoLoader:
    def __init__(self, file_path, max_buffered_frames=100):
        self.video=cv2.VideoCapture(file_path)

        fpc=self.video.get(cv2.CAP_PROP_FRAME_COUNT) 
        fps = self.video.get(cv2.CAP_PROP_FPS)
        duration = fpc/fps
        self.delay=duration/fpc 
        print(f'fps={fps}, fpc={fpc}  duration={duration} seconds delay={self.delay} seconds')

        self.buffer=deque(maxlen=max_buffered_frames)
        while(self.video.isOpened()):
            ret, frame = self.video.read()
            self.buffer.append(frame)
            break 
            # if ret==False: break
            # self.buffer.append(frame)
            # if len(self.buffer)==self.buffer.maxlen: 
            #     break 

        self.current_frame_id=0
        self.count=-1

    def current_frame(self):
        return self.buffer[self.current_frame_id]
    
    def next_frame(self):
        self.current_frame_id+=1
        self.count+=1

        if self.current_frame_id<len(self.buffer): # buffer has the frame
            return True
        elif len(self.buffer)<self.buffer.maxlen: # buffer is not fully loaded
            ret=False
            if self.video.isOpened():
                ret, frame = self.video.read()
            
            if ret==False: return False
            self.buffer.append(frame) 
            
            return True
        else:                                      # buffer is full and requested frame is ahead of buffer
            ret=False
            if self.video.isOpened():
                ret, frame = self.video.read()
            
            if ret==False: return False
            self.buffer.append(frame)
            self.current_frame_id-=1               # move back the current frame id
            return True
        
    def prev_frame(self):
        if self.current_frame_id>0:
            self.current_frame_id-=1
            self.count-=1
            return True 
        else:
            return False
            

    def close(self):
        self.video.release()

In [19]:
vl=VideoLoader(file_path, max_buffered_frames=500)

closed=False
key_pressed=None 
paused=False

frame_ids=[]

while True:
    img=vl.current_frame().copy()

    if vl.current_frame().shape[0]>700:
        w,h,_=vl.current_frame().shape
        scale=0.5
        img=cv2.resize(vl.current_frame(), (int(h*scale), int(w*scale)))

    # frame_no=vl.current_frame_id
    frame_no=vl.count

    if frame_no==1: paused=True


    font = cv2.FONT_HERSHEY_SIMPLEX
    text = 'Frame: ' + str(frame_no)
    cv2.putText(img, text, (10, 50), font, 1, (0, 255, 255), 2, cv2.LINE_AA)

    info=f"space: pause/play \nq: quit \na: previous frame \nd: next frame\ns: save frame\nty: -+t\ndelay: {vl.delay:.4f} seconds"
    y0, dy = 50, 20
    for i, line in enumerate(info.split('\n')):
        y = y0 + i*dy
        cv2.putText(img, line, (200, y ), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        # cv2.putText(img, line, (100, y ), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)


    cv2.imshow('Video', img)
    #get key pressed
    key_pressed=cv2.waitKey(1)

    
    if not paused and key_pressed & 0xFF == ord(' '):
        paused=True 
    elif paused and key_pressed & 0xFF == ord(' '):
        paused=False
    elif key_pressed & 0xFF == ord('a'):
        if not vl.prev_frame(): break  
        
    elif key_pressed & 0xFF == ord('d'):
        if not vl.next_frame(): break
    
    elif key_pressed & 0xFF == ord('t'):
        vl.delay -= 0.005
        if vl.delay <0.001: vl.delay=0.001
        
    elif key_pressed & 0xFF == ord('y'):
        vl.delay += 0.005

    elif key_pressed & 0xFF == ord('s'):
        frame_ids.append(frame_no)
        print(f'frame_ids={frame_ids}')

    if key_pressed & 0xFF == ord('q'): 
        break 

    if not paused:
        if not vl.next_frame(): break
    time.sleep(vl.delay)

closed=True 
cv2.destroyAllWindows()  

fps=30.0, fpc=3342.0  duration=111.4 seconds delay=0.03333333333333333 seconds
frame_ids=[281]
frame_ids=[281, 348]
frame_ids=[281, 348, 772]
frame_ids=[281, 348, 772, 820]
frame_ids=[281, 348, 772, 820, 1163]
frame_ids=[281, 348, 772, 820, 1163, 1232]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332, 2628]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332, 2628, 2683]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332, 2628, 2683, 2987]
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332, 2628, 2683,

In [20]:
len(frame_ids)

16

In [23]:
frame_ids=[281, 348, 772, 820, 1163, 1232, 1504, 1549, 1962, 2004, 2273, 2332, 2628, 2683, 2987, 3033]
len(frame_ids)

16